In [1]:
from typing import List

def show(board):
    for i in range(0, len(board)):
        for j in range(0, len(board[i])):
            print(board[i][j] , " ", end="")
        print("\n")

In [2]:
#with cache (right but slow)
class Solution:
    def split(self, word):
        return [char for char in word]

    def initial_indexes(self, board, c):
        result = []
        for i in range(0, len(board)):
            for j in range(0, len(board[0])):
                if board[i][j] == c:
                    result.append((i,j))
        return result

    def letters(self, board):
        result = {}
        for i in range(0, len(board)):
            for j in range(0, len(board[0])):
                if board[i][j] not in result:
                    result[board[i][j]] = None
        return result

    def exist(self, index: tuple, taken: dict, board: List[List[str]], letter: str) -> dict:
        posibilities = []
        n = len(board)
        m = len(board[0])
        i = index[0]
        j = index[1]
        if i + 1 < n:
            if board[i + 1][j] == letter and (i+1,j) not in taken:
                posibilities.append((i+1,j))
        if not i - 1 < 0:
            if board[i - 1][j] == letter and (i-1,j) not in taken:
                posibilities.append((i-1,j))
        if j + 1 < m:
            if board[i][j+1] == letter and (i,j+1) not in taken:
                posibilities.append((i,j+1))
        if not j - 1 < 0:
            if board[i][j-1] == letter and (i,j-1) not in taken:
                posibilities.append((i,j-1))
        return posibilities

    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        result = []
        cache = {}
        letters = self.letters(board)
        for word in words:
            if word[0] not in letters or word[-1] not in letters:
                continue
            index_ = 1
            if word in cache:
                result.append(word)
                continue
            #print('------->', word)
            #find initials
            if not word[0] in cache:
                indexes = self.initial_indexes(board,  word[0])
                if len(indexes) == 0:
                    #no indexes were found
                    continue
                l = []
                for index in indexes:
                    l.append({index:None})
                cache[word[0]] = l
            for i in range(1, len(word) -1):
                if word[: len(word) - i] in cache:
                    #print('current', word[-i])
                    #print('found', word[: len(word) - i], 'in cache')
                    #print('full word would be', word[:len(word) - i + 1])
                    l = []
                    #print(cache)
                    #print(cache[word[:i]])
                    for chache_element in cache[word[: len(word) - i]]:
                        #print('cache for',word[0: len(word) - i],chache_element) 
                        #print('current letter:',word[i])
                        posibilities = self.exist(list(chache_element)[-1], chache_element, board, word[-i])
                        #print('possinbilities for', word[-i])
                        for posibility in posibilities:
                            #print(posibility)
                            new_cache = chache_element.copy()
                            new_cache[posibility] = None
                            l.append(new_cache)
                    if len(l) > 0:
                        if not word[:len(word) - i + 1] in cache:
                            cache[word[:len(word) - i + 1]] = l
                    index_ = len(word) - i
                    break
            for i in range(index_, len(word)):
                if word[:i] in cache:
                    l = []
                    #print(cache[word[:i]])
                    for chache_element in cache[word[:i]]:
                        #print('cache for',word[:i],chache_element) 
                        #print('current letter:',word[i])
                        posibilities = self.exist(list(chache_element)[-1], chache_element, board, word[i])
                        #print('possinbilities for', word[i])
                        for posibility in posibilities:
                            #print(posibility)
                            new_cache = chache_element.copy()
                            new_cache[posibility] = None
                            l.append(new_cache)
                    if len(l) > 0:
                        if not word[:i+1] in cache:
                            cache[word[:i+1]] = l
                    continue
                break     
            if word in cache:
                result.append(word)
        #for key in cache:
            #print(key, cache[key])
        return result


In [4]:
#naiv without copy (right but slow)
class Solution:
    def split(self, word):
        return [char for char in word]

    def letters(self, board):
        result = {}
        for i in range(0, len(board)):
            for j in range(0, len(board[0])):
                if board[i][j] not in result:
                    result[board[i][j]] = None
        return result

    def exist(self, board: List[List[str]], word: str) -> bool:
        if len(word) < 1:
            return True
        n = len(board)
        m = len(board[0])
        if len(word) > n*m:
            return False
        seen = {}
        def test_dude(dude, letters) -> bool:
            if len(letters) < 1:
                return True
            i = dude[0]
            j = dude[1]
            letters_ = letters.copy() 
            if i + 1 < n:
                if board[i+1][j] == letters_[0] and (i+1,j) not in seen:
                    seen[(i+1,j)] = None
                    letters_.remove(board[i+1][j])
                    if test_dude((i+1,j), letters_):
                        return True
                    letters_.insert(0,board[i+1][j])
                    del seen[(i+1,j)]
            if not i - 1 < 0:
                if board[i-1][j] == letters_[0] and (i-1,j) not in seen:
                    seen[(i-1,j)] = None
                    letters_.remove(board[i-1][j])
                    if test_dude((i-1,j), letters_):
                        return True
                    letters_.insert(0,board[i-1][j])
                    del seen[(i-1,j)]
            if j + 1 < m:
                if board[i][j+1] == letters_[0] and (i,j+1) not in seen:
                    seen[(i,j+1)] = None
                    letters_.remove(board[i][j+1])
                    if test_dude((i,j+1), letters_):
                        return True
                    letters_.insert(0,board[i][j+1])
                    del seen[(i,j+1)]
            if not j - 1 < 0:
                if board[i][j-1] == letters_[0] and (i,j-1) not in seen:
                    seen[(i,j-1)] = None
                    letters_.remove(board[i][j-1])
                    if test_dude((i,j-1), letters_):
                        return True
                    letters_.insert(0,board[i][j-1])
                    del seen[(i,j-1)]
            return False

        letters = self.split(word)
        for i in range(0, n):
            for j in range(0, m):
                letters_ = letters.copy()
                if board[i][j] == letters_[0]:
                    seen[(i,j)] = None
                    letters_.remove(board[i][j])
                    if len(letters_) < 1:
                        return True
                    if i + 1 < n:
                        if board[i+1][j] == letters_[0] and (i+1,j) not in seen:
                            seen[(i+1,j)] = None
                            letters_.remove(board[i+1][j])
                            if test_dude((i+1,j), letters_):
                                return True
                            letters_.insert(0,board[i+1][j])
                            del seen[(i+1,j)]
                    if not i - 1 < 0:
                        if board[i-1][j] == letters_[0] and (i-1,j) not in seen:
                            seen[(i-1,j)] = None
                            letters_.remove(board[i-1][j])
                            if test_dude((i-1,j), letters_):
                                return True
                            letters_.insert(0,board[i-1][j])
                            del seen[(i-1,j)]
                    if j + 1 < m:
                        if board[i][j+1] == letters_[0] and (i,j+1) not in seen:
                            seen[(i,j+1)] = None
                            letters_.remove(board[i][j+1])
                            if test_dude((i,j+1), letters_):
                                return True
                            letters_.insert(0,board[i][j+1])
                            del seen[(i,j+1)]
                    if not j - 1 < 0:
                        if board[i][j-1] == letters_[0] and (i,j-1) not in seen:
                            seen[(i,j-1)] = None
                            letters_.remove(board[i][j-1])
                            if test_dude((i,j-1), letters_):
                                return True
                            letters_.insert(0,board[i][j-1])
                            del seen[(i,j-1)]
                    del seen[(i,j)]
        return False

    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        result = []
        letters = self.letters(board)
        for i in range(0, len(words)):
            if words[i][0] not in letters or words[i][-1] not in letters:
                continue
            if self.exist(board, words[i]):
                result.append(words[i])
        return result

In [4]:
#internet
class TrieNode:
    def __init__(self, char=None, letterToChild=None, word=None, prev=None):
        self.char = char # this node's current character
        self.letterToChild = letterToChild # dictionary that maps child's letter to child's node
        self.word = word #  when a word is complete from start to this node, we set what the word is from start to here (note that there can stll be more words down this path)
        self.prev = prev # to backtrack and remove words that are already searched

class Solution:
    #24ms (faster than 98.86% Python 3) ~ O(MN * numPaths * 3^(longestWordLength)) runtime, O(totalWordLength * numWords) worst case memory
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        if not board or len(board[0]) == 0 or not words:
            return []

        def buildTrie(words: List[str]):
            #print("using words: {}".format(words))
            # root node is character "/"
            root = TrieNode(char="/")
            for word in words:
                # given a word, construct path of the word in Trie
                index = 0
                curNode = root
                while index < len(word):
                    #print("word[index]: {}".format(word[index]))
                    # prepare childNode
                    char = word[index]
                    if not curNode.letterToChild:
                        curNode.letterToChild = {}
                    if char not in curNode.letterToChild:
                        # if childNode doesn't exist, create it
                        childNode = TrieNode(char=char,letterToChild={},prev=curNode) #for some reason I need the {} otherwise it's stealing from previous node
                        curNode.letterToChild[char] = childNode
                        #print("curNode's child: {}".format(curNode.letterToChild))
                    else:
                        childNode = curNode.letterToChild[char]
                    
                    # if at end of word, update the child node
                    if index == len(word) - 1:
                        childNode.word = word
                        break

                    index += 1
                    curNode = childNode
            return root
        #build the Trie
        root = buildTrie(words)

        #iterate through the board
        rows = len(board)
        cols = len(board[0])
        
        resultList = []
        def dfs(x,y,curNode:TrieNode):
            #print("(x,y)=({},{})".format(x,y))
            boardLetter = board[x][y]
            if boardLetter in curNode.letterToChild:
                board[x][y] = "*" #modify the board letter so we don't reuse during dfs search
                # letter matches, get the childNode
                nextNode = curNode.letterToChild[boardLetter]
                # check if nextNode is at the end of a word; if so, add that to our resultList. Note that there may be more words further along this path.
                if nextNode.word:
                    resultList.append(nextNode.word)
                    nextNode.word = None
                    # delete the branch that is no longer needed (to avoid creating the same word again on next board iteration)
                    if len(nextNode.letterToChild) == 0:
                        #no more children below so need to backtrack and delete
                        #backtrack stops if there are 2+ paths down, or if we're at the root node
                        parentNode = curNode
                        usedChar = nextNode.char
                        while len(parentNode.letterToChild) == 1:
                            #cut mapping of parentNode to used-childNode
                            del parentNode.letterToChild[usedChar]
                            usedChar = parentNode.char
                            if usedChar == "/":
                                #reached the top so no need to backtrack anymore
                                break
                            parentNode = parentNode.prev                            

                # keep dfs traversing until end of word(s) in this Trie path
                for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
                    adjX, adjY = x+dx, y+dy 
                    # check if adjacent position is in board
                    if 0 <= adjX < rows and 0 <= adjY < cols:
                        dfs(adjX,adjY,nextNode)
            # no match with board or no more letters along this path
            # revert back the letter modified by dfs (backtrack)
            board[x][y] = boardLetter
            return

        for x in range(rows):
            for y in range(cols):
                #check if word match; if so, traverse dfs
                curNode = root 
                dfs(x,y,curNode)
        return resultList

In [5]:
board = [["a","b","c"],["a","e","d"],["a","f","g"]]
words = ["abcdef","abcdefg","gfedcbaaa","eaabcdgfa","eaabcdgf","befa","dgc","ade", "abc"]
board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]]
words = ["oath","pea","eat","rain"]
board = [["a"]]
words = ["b"]
board = [["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"]]
words = ["ababababaa","ababababab","ababababac","ababababad","ababababae","ababababaf","ababababag","ababababah","ababababai","ababababaj","ababababak","ababababal","ababababam","ababababan","ababababao","ababababap","ababababaq","ababababar","ababababas","ababababat","ababababau","ababababav","ababababaw","ababababax","ababababay","ababababaz","ababababba","ababababbb","ababababbc","ababababbd","ababababbe","ababababbf","ababababbg","ababababbh","ababababbi","ababababbj","ababababbk","ababababbl","ababababbm","ababababbn","ababababbo","ababababbp","ababababbq","ababababbr","ababababbs","ababababbt","ababababbu","ababababbv","ababababbw","ababababbx","ababababby","ababababbz","ababababca","ababababcb","ababababcc","ababababcd","ababababce","ababababcf","ababababcg","ababababch","ababababci","ababababcj","ababababck","ababababcl","ababababcm","ababababcn","ababababco","ababababcp","ababababcq","ababababcr","ababababcs","ababababct","ababababcu","ababababcv","ababababcw","ababababcx","ababababcy","ababababcz","ababababda","ababababdb","ababababdc","ababababdd","ababababde","ababababdf","ababababdg","ababababdh","ababababdi","ababababdj","ababababdk","ababababdl","ababababdm","ababababdn","ababababdo","ababababdp","ababababdq","ababababdr","ababababds","ababababdt","ababababdu","ababababdv","ababababdw","ababababdx","ababababdy","ababababdz","ababababea","ababababeb","ababababec","ababababed","ababababee","ababababef","ababababeg","ababababeh","ababababei","ababababej","ababababek","ababababel","ababababem","ababababen","ababababeo","ababababep","ababababeq","ababababer","ababababes","ababababet","ababababeu","ababababev","ababababew","ababababex","ababababey","ababababez","ababababfa","ababababfb","ababababfc","ababababfd","ababababfe","ababababff","ababababfg","ababababfh","ababababfi","ababababfj","ababababfk","ababababfl","ababababfm","ababababfn","ababababfo","ababababfp","ababababfq","ababababfr","ababababfs","ababababft","ababababfu","ababababfv","ababababfw","ababababfx","ababababfy","ababababfz","ababababga","ababababgb","ababababgc","ababababgd","ababababge","ababababgf","ababababgg","ababababgh","ababababgi","ababababgj","ababababgk","ababababgl","ababababgm","ababababgn","ababababgo","ababababgp","ababababgq","ababababgr","ababababgs","ababababgt","ababababgu","ababababgv","ababababgw","ababababgx","ababababgy","ababababgz","ababababha","ababababhb","ababababhc","ababababhd","ababababhe","ababababhf","ababababhg","ababababhh","ababababhi","ababababhj","ababababhk","ababababhl","ababababhm","ababababhn","ababababho","ababababhp","ababababhq","ababababhr","ababababhs","ababababht","ababababhu","ababababhv","ababababhw","ababababhx","ababababhy","ababababhz","ababababia","ababababib","ababababic","ababababid","ababababie","ababababif","ababababig","ababababih","ababababii","ababababij","ababababik","ababababil","ababababim","ababababin","ababababio","ababababip","ababababiq","ababababir","ababababis","ababababit","ababababiu","ababababiv","ababababiw","ababababix","ababababiy","ababababiz","ababababja","ababababjb","ababababjc","ababababjd","ababababje","ababababjf","ababababjg","ababababjh","ababababji","ababababjj","ababababjk","ababababjl","ababababjm","ababababjn","ababababjo","ababababjp","ababababjq","ababababjr","ababababjs","ababababjt","ababababju","ababababjv","ababababjw","ababababjx","ababababjy","ababababjz","ababababka","ababababkb","ababababkc","ababababkd","ababababke","ababababkf","ababababkg","ababababkh","ababababki","ababababkj","ababababkk","ababababkl","ababababkm","ababababkn","ababababko","ababababkp","ababababkq","ababababkr","ababababks","ababababkt","ababababku","ababababkv","ababababkw","ababababkx","ababababky","ababababkz","ababababla","abababablb","abababablc","ababababld","abababable","abababablf","abababablg","abababablh","ababababli","abababablj","abababablk","ababababll","abababablm","ababababln","abababablo","abababablp","abababablq","abababablr","ababababls","abababablt","abababablu","abababablv","abababablw","abababablx","abababably","abababablz","ababababma","ababababmb","ababababmc","ababababmd","ababababme","ababababmf","ababababmg","ababababmh","ababababmi","ababababmj","ababababmk","ababababml","ababababmm","ababababmn","ababababmo","ababababmp","ababababmq","ababababmr","ababababms","ababababmt","ababababmu","ababababmv","ababababmw","ababababmx","ababababmy","ababababmz","ababababna","ababababnb","ababababnc","ababababnd","ababababne","ababababnf","ababababng","ababababnh","ababababni","ababababnj","ababababnk","ababababnl","ababababnm","ababababnn","ababababno","ababababnp","ababababnq","ababababnr","ababababns","ababababnt","ababababnu","ababababnv","ababababnw","ababababnx","ababababny","ababababnz","ababababoa","ababababob","ababababoc","ababababod","ababababoe","ababababof","ababababog","ababababoh","ababababoi","ababababoj","ababababok","ababababol","ababababom","ababababon","ababababoo","ababababop","ababababoq","ababababor","ababababos","ababababot","ababababou","ababababov","ababababow","ababababox","ababababoy","ababababoz","ababababpa","ababababpb","ababababpc","ababababpd","ababababpe","ababababpf","ababababpg","ababababph","ababababpi","ababababpj","ababababpk","ababababpl","ababababpm","ababababpn","ababababpo","ababababpp","ababababpq","ababababpr","ababababps","ababababpt","ababababpu","ababababpv","ababababpw","ababababpx","ababababpy","ababababpz","ababababqa","ababababqb","ababababqc","ababababqd","ababababqe","ababababqf","ababababqg","ababababqh","ababababqi","ababababqj","ababababqk","ababababql","ababababqm","ababababqn","ababababqo","ababababqp","ababababqq","ababababqr","ababababqs","ababababqt","ababababqu","ababababqv","ababababqw","ababababqx","ababababqy","ababababqz","ababababra","ababababrb","ababababrc","ababababrd","ababababre","ababababrf","ababababrg","ababababrh","ababababri","ababababrj","ababababrk","ababababrl","ababababrm","ababababrn","ababababro","ababababrp","ababababrq","ababababrr","ababababrs","ababababrt","ababababru","ababababrv","ababababrw","ababababrx","ababababry","ababababrz","ababababsa","ababababsb","ababababsc","ababababsd","ababababse","ababababsf","ababababsg","ababababsh","ababababsi","ababababsj","ababababsk","ababababsl","ababababsm","ababababsn","ababababso","ababababsp","ababababsq","ababababsr","ababababss","ababababst","ababababsu","ababababsv","ababababsw","ababababsx","ababababsy","ababababsz","ababababta","ababababtb","ababababtc","ababababtd","ababababte","ababababtf","ababababtg","ababababth","ababababti","ababababtj","ababababtk","ababababtl","ababababtm","ababababtn","ababababto","ababababtp","ababababtq","ababababtr","ababababts","ababababtt","ababababtu","ababababtv","ababababtw","ababababtx","ababababty","ababababtz","ababababua","ababababub","ababababuc","ababababud","ababababue","ababababuf","ababababug","ababababuh","ababababui","ababababuj","ababababuk","ababababul","ababababum","ababababun","ababababuo","ababababup","ababababuq","ababababur","ababababus","ababababut","ababababuu","ababababuv","ababababuw","ababababux","ababababuy","ababababuz","ababababva","ababababvb","ababababvc","ababababvd","ababababve","ababababvf","ababababvg","ababababvh","ababababvi","ababababvj","ababababvk","ababababvl","ababababvm","ababababvn","ababababvo","ababababvp","ababababvq","ababababvr","ababababvs","ababababvt","ababababvu","ababababvv","ababababvw","ababababvx","ababababvy","ababababvz","ababababwa","ababababwb","ababababwc","ababababwd","ababababwe","ababababwf","ababababwg","ababababwh","ababababwi","ababababwj","ababababwk","ababababwl","ababababwm","ababababwn","ababababwo","ababababwp","ababababwq","ababababwr","ababababws","ababababwt","ababababwu","ababababwv","ababababww","ababababwx","ababababwy","ababababwz","ababababxa","ababababxb","ababababxc","ababababxd","ababababxe","ababababxf","ababababxg","ababababxh","ababababxi","ababababxj","ababababxk","ababababxl","ababababxm","ababababxn","ababababxo","ababababxp","ababababxq","ababababxr","ababababxs","ababababxt","ababababxu","ababababxv","ababababxw","ababababxx","ababababxy","ababababxz","ababababya","ababababyb","ababababyc","ababababyd","ababababye","ababababyf","ababababyg","ababababyh","ababababyi","ababababyj","ababababyk","ababababyl","ababababym","ababababyn","ababababyo","ababababyp","ababababyq","ababababyr","ababababys","ababababyt","ababababyu","ababababyv","ababababyw","ababababyx","ababababyy","ababababyz","ababababza","ababababzb","ababababzc","ababababzd","ababababze","ababababzf","ababababzg","ababababzh","ababababzi","ababababzj","ababababzk","ababababzl","ababababzm","ababababzn","ababababzo","ababababzp","ababababzq","ababababzr","ababababzs","ababababzt","ababababzu","ababababzv","ababababzw","ababababzx","ababababzy","ababababzz"]

print(Solution().findWords(board, words))

['ababababab']
